In [4]:
import random
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score  

In [6]:
#Run the code for Origial Dataset
# Load and prepare data
data = pd.read_csv("synth_seg.csv")
y = data['decision'].astype(bool)
X = data.drop(columns=['Subject', 'decision', 'neuropsych_score'], axis=1)
n_features = X.shape[1]

# Genetic Algorithm Functions From Scratch
def create_individual(n_features):
    # Creates a random individual (feature subset).
    return [random.randint(0, 1) for _ in range(n_features)]

def apply_lda(numeric_df, y, score):
    n_components = 1
    lda = LDA(n_components=n_components)

    # Remove train_test_split, we will use cross-validation directly on the entire dataset
    classifier = LogisticRegression(max_iter=1000)

    skf = StratifiedKFold(n_splits=5)
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in skf.split(numeric_df, y):
        X_train_fold, X_test_fold = numeric_df.iloc[train_index], numeric_df.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        X_lda = lda.fit_transform(X_train_fold, y_train_fold)
        lda_df = pd.DataFrame(data=X_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        classifier.fit(lda_df, y_train_fold)

        X_test_lda = lda.transform(X_test_fold)
        X_test_df =  pd.DataFrame(data=X_test_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        y_pred_fold = classifier.predict(X_test_df)

        precision_scores.append(precision_score(y_test_fold, y_pred_fold, average='macro'))
        recall_scores.append(recall_score(y_test_fold, y_pred_fold, average='macro'))
        accuracy_scores.append(classifier.score(X_test_df, y_test_fold))
        different_locs = np.where(y_test_fold != y_pred_fold)[0]

        # Map these back to the original indices in lda_df
        original_different_locs = test_index[different_locs]

    return sum(accuracy_scores) / len(accuracy_scores)

def evaluate(individual, X, y):
    # Evaluates the fitness of an individual.
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:
        return 0  # Avoid selecting no features

    X_selected = X.iloc[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)

    # Apply LDA using the apply_lda function
    accuracy_lda = apply_lda(pd.DataFrame(X_scaled), y, data['neuropsych_score'])  # Pass scaled data as DataFrame

    return accuracy_lda  # Use only LDA accuracy here


def crossover(parent1, parent2):
    # Performs crossover at a random point.
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


def mutation(individual, indpb=0.05):
    # Performs flip-bit mutation at a single random point.
    if random.random() < indpb:  # Check if mutation should occur
        point = random.randint(0, len(individual) - 1)  # Select a random point
        individual[point] = 1 - individual[point]  # Flip the bit at the selected point
    return (individual,)


# Main Genetic Algorithm Call Function
def main(X, y, pop_size=100, ngen=10, cxpb=0.5, mutpb=0.2, top_percent=0.10, tolerance=1e-4):
    random.seed(42)

    # Initialize population
    population = [create_individual(n_features) for _ in range(pop_size)]

    # Evaluate initial population
    fitnesses = [evaluate(ind, X, y) for ind in population]

    # Store best individuals, their generation, and their fitness values
    best_individuals = []
    best_generations = []
    best_fitnesses = []

    for gen in range(ngen):
        # Selection (Select top individuals directly)
        num_top = int(pop_size * top_percent)
        top_indices = np.argsort(fitnesses)[-num_top:]
        offspring = [population[i] for i in top_indices]

        # Crossover to create the rest of the offspring
        num_crossovers = pop_size - num_top  # Number of crossovers needed
        for _ in range(num_crossovers // 2):  # // 2 because each crossover creates 2 offspring
            parent1, parent2 = random.sample(offspring, 2)  # Select parents from the top individuals
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        # Mutation
        for i in range(num_top, len(offspring)):  # Mutate only the new offspring
            if random.random() < mutpb:
                offspring[i], = mutation(offspring[i])

        # Evaluate offspring
        fitnesses = [evaluate(ind, X, y) for ind in offspring]

        # Replace population with offspring (Elitism: Keep top individuals)
        population[:] = offspring  

        # Store the top individuals and their generation
        top_indices = np.argsort(fitnesses)[-num_top:]
        for i in top_indices:
            best_individuals.append(population[i])
            best_generations.append(gen)  # Store the generation

        # Store fitness of the best individual in the current generation
        best_fitnesses.append(max(fitnesses))

        # Convergence check
        if gen > 1 and abs(best_fitnesses[-1] - best_fitnesses[-2]) < tolerance:
            print(f"Converged at generation {gen}")
            break

    return best_individuals, best_generations, best_fitnesses  # Return all three lists

# Run the GA
best_individuals, best_generations, best_fitnesses = main(X, y, pop_size=100, top_percent=0.10)  # Use top 10%

# Print the final best solution and its generation
best_individual = max(best_individuals, key=lambda ind: evaluate(ind, X, y))
best_index = best_individuals.index(best_individual)
best_generation = best_generations[best_index]

selected_features_lda = [i for i, bit in enumerate(best_individual) if bit == 1]
print(f"Final Best Solution (found at generation {best_generation}):")
print(f"  Selected Features: {X.columns[selected_features_lda].tolist()}")
print(f"  Number of features selected: {len(selected_features_lda)}")

# Print fitness values across generations
print("\nFitness values across generations:")
for gen, fitness in enumerate(best_fitnesses):
    print(f"Generation {gen}: {fitness}")

Converged at generation 2
Final Best Solution (found at generation 1):
  Selected Features: ['general white matter', 'general grey matter', 'general csf', 'cerebellum', 'brainstem', 'thalamus', 'hippocampus+amygdala', 'left lateral ventricle', 'left inferior lateral ventricle', 'left cerebellum white matter', 'left thalamus', 'left putamen', 'left hippocampus', 'left amygdala', 'right inferior lateral ventricle', 'right accumbens area', 'ctx-lh-bankssts', 'ctx-lh-caudalmiddlefrontal', 'ctx-lh-cuneus', 'ctx-lh-entorhinal', 'ctx-lh-inferiorparietal', 'ctx-lh-inferiortemporal', 'ctx-lh-lateralorbitofrontal', 'ctx-lh-lingual', 'ctx-lh-parsopercularis', 'ctx-lh-parstriangularis', 'ctx-lh-postcentral', 'ctx-lh-posteriorcingulate', 'ctx-lh-precuneus', 'ctx-lh-superiorfrontal', 'ctx-lh-superiortemporal', 'ctx-lh-supramarginal', 'ctx-lh-temporalpole', 'ctx-lh-insula', 'ctx-rh-bankssts', 'ctx-rh-caudalanteriorcingulate', 'ctx-rh-cuneus', 'ctx-rh-entorhinal', 'ctx-rh-inferiorparietal', 'ctx-rh-in

In [7]:
#Run code for the Over Sampled Data
# Load and prepare data
data = pd.read_csv("resampled_data.csv")
y = data['decision'].astype(bool)
X = data.drop(columns=['Subject', 'decision', 'neuropsych_score'], axis=1)
n_features = X.shape[1]

# Genetic Algorithm Functions From Scratch
def create_individual(n_features):
    # Creates a random individual (feature subset).
    return [random.randint(0, 1) for _ in range(n_features)]

def apply_lda(numeric_df, y, score):
    n_components = 1
    lda = LDA(n_components=n_components)

    # Remove train_test_split, we will use cross-validation directly on the entire dataset
    classifier = LogisticRegression(max_iter=1000)

    skf = StratifiedKFold(n_splits=5)
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in skf.split(numeric_df, y):
        X_train_fold, X_test_fold = numeric_df.iloc[train_index], numeric_df.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        X_lda = lda.fit_transform(X_train_fold, y_train_fold)
        lda_df = pd.DataFrame(data=X_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        classifier.fit(lda_df, y_train_fold)

        X_test_lda = lda.transform(X_test_fold)
        X_test_df =  pd.DataFrame(data=X_test_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        y_pred_fold = classifier.predict(X_test_df)

        precision_scores.append(precision_score(y_test_fold, y_pred_fold, average='macro'))
        recall_scores.append(recall_score(y_test_fold, y_pred_fold, average='macro'))
        accuracy_scores.append(classifier.score(X_test_df, y_test_fold))
        different_locs = np.where(y_test_fold != y_pred_fold)[0]

        # Map these back to the original indices in lda_df
        original_different_locs = test_index[different_locs]

    return sum(accuracy_scores) / len(accuracy_scores)

def evaluate(individual, X, y):
    # Evaluates the fitness of an individual.
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:
        return 0  # Avoid selecting no features

    X_selected = X.iloc[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)

    # Apply LDA using the apply_lda function
    accuracy_lda = apply_lda(pd.DataFrame(X_scaled), y, data['neuropsych_score'])  # Pass scaled data as DataFrame

    return accuracy_lda  # Use only LDA accuracy here


def crossover(parent1, parent2):
    # Performs crossover at a random point.
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


def mutation(individual, indpb=0.05):
    # Performs flip-bit mutation at a single random point.
    if random.random() < indpb:  # Check if mutation should occur
        point = random.randint(0, len(individual) - 1)  # Select a random point
        individual[point] = 1 - individual[point]  # Flip the bit at the selected point
    return (individual,)


# Main Genetic Algorithm Call Function
def main(X, y, pop_size=100, ngen=10, cxpb=0.5, mutpb=0.2, top_percent=0.10, tolerance=1e-4):
    random.seed(42)

    # Initialize population
    population = [create_individual(n_features) for _ in range(pop_size)]

    # Evaluate initial population
    fitnesses = [evaluate(ind, X, y) for ind in population]

    # Store best individuals, their generation, and their fitness values
    best_individuals = []
    best_generations = []
    best_fitnesses = []

    for gen in range(ngen):
        # Selection (Select top individuals directly)
        num_top = int(pop_size * top_percent)
        top_indices = np.argsort(fitnesses)[-num_top:]
        offspring = [population[i] for i in top_indices]

        # Crossover to create the rest of the offspring
        num_crossovers = pop_size - num_top  # Number of crossovers needed
        for _ in range(num_crossovers // 2):  # // 2 because each crossover creates 2 offspring
            parent1, parent2 = random.sample(offspring, 2)  # Select parents from the top individuals
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        # Mutation
        for i in range(num_top, len(offspring)):  # Mutate only the new offspring
            if random.random() < mutpb:
                offspring[i], = mutation(offspring[i])

        # Evaluate offspring
        fitnesses = [evaluate(ind, X, y) for ind in offspring]

        # Replace population with offspring (Elitism: Keep top individuals)
        population[:] = offspring  

        # Store the top individuals and their generation
        top_indices = np.argsort(fitnesses)[-num_top:]
        for i in top_indices:
            best_individuals.append(population[i])
            best_generations.append(gen)  # Store the generation

        # Store fitness of the best individual in the current generation
        best_fitnesses.append(max(fitnesses))

        # Convergence check
        if gen > 1 and abs(best_fitnesses[-1] - best_fitnesses[-2]) < tolerance:
            print(f"Converged at generation {gen}")
            break

    return best_individuals, best_generations, best_fitnesses  # Return all three lists

# Run the GA
best_individuals, best_generations, best_fitnesses = main(X, y, pop_size=100, top_percent=0.10)  # Use top 10%

# Print the final best solution and its generation
best_individual = max(best_individuals, key=lambda ind: evaluate(ind, X, y))
best_index = best_individuals.index(best_individual)
best_generation = best_generations[best_index]

selected_features_lda = [i for i, bit in enumerate(best_individual) if bit == 1]
print(f"Final Best Solution (found at generation {best_generation}):")
print(f"  Selected Features: {X.columns[selected_features_lda].tolist()}")
print(f"  Number of features selected: {len(selected_features_lda)}")

# Print fitness values across generations
print("\nFitness values across generations:")
for gen, fitness in enumerate(best_fitnesses):
    print(f"Generation {gen}: {fitness}")

Converged at generation 2
Final Best Solution (found at generation 1):
  Selected Features: ['general white matter', 'general grey matter', 'general csf', 'cerebellum', 'brainstem', 'thalamus', 'putamen+pallidum', 'left cerebral cortex', 'left lateral ventricle', 'left inferior lateral ventricle', 'left cerebellum white matter', 'left thalamus', 'left pallidum', '3rd ventricle', 'brain-stem', 'left hippocampus', 'csf', 'left accumbens area', 'left ventral DC', 'right lateral ventricle', 'right inferior lateral ventricle', 'right cerebellum cortex', 'right caudate', 'right putamen', 'right pallidum', 'right hippocampus', 'right amygdala', 'right accumbens area', 'right ventral DC', 'ctx-lh-bankssts', 'ctx-lh-caudalanteriorcingulate', 'ctx-lh-cuneus', 'ctx-lh-entorhinal', 'ctx-lh-inferiorparietal', 'ctx-lh-isthmuscingulate', 'ctx-lh-lateraloccipital', 'ctx-lh-lateralorbitofrontal', 'ctx-lh-paracentral', 'ctx-lh-pericalcarine', 'ctx-lh-postcentral', 'ctx-lh-posteriorcingulate', 'ctx-lh-pr

In [9]:
#Run the Code for Under Sampled Dataset
# Load and prepare data
data = pd.read_csv("undersampled_data.csv")
y = data['decision'].astype(bool)
X = data.drop(columns=['Subject', 'decision', 'neuropsych_score'], axis=1)
n_features = X.shape[1]

# Genetic Algorithm Functions From Scratch
def create_individual(n_features):
    # Creates a random individual (feature subset).
    return [random.randint(0, 1) for _ in range(n_features)]

def apply_lda(numeric_df, y, score):
    n_components = 1
    lda = LDA(n_components=n_components)

    # Remove train_test_split, we will use cross-validation directly on the entire dataset
    classifier = LogisticRegression(max_iter=1000)

    skf = StratifiedKFold(n_splits=5)
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in skf.split(numeric_df, y):
        X_train_fold, X_test_fold = numeric_df.iloc[train_index], numeric_df.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        X_lda = lda.fit_transform(X_train_fold, y_train_fold)
        lda_df = pd.DataFrame(data=X_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        classifier.fit(lda_df, y_train_fold)

        X_test_lda = lda.transform(X_test_fold)
        X_test_df =  pd.DataFrame(data=X_test_lda, columns=[f'LD{i+1}' for i in range(n_components)])
        y_pred_fold = classifier.predict(X_test_df)

        precision_scores.append(precision_score(y_test_fold, y_pred_fold, average='macro'))
        recall_scores.append(recall_score(y_test_fold, y_pred_fold, average='macro'))
        accuracy_scores.append(classifier.score(X_test_df, y_test_fold))
        different_locs = np.where(y_test_fold != y_pred_fold)[0]

        # Map these back to the original indices in lda_df
        original_different_locs = test_index[different_locs]

    return sum(accuracy_scores) / len(accuracy_scores)

def evaluate(individual, X, y):
    # Evaluates the fitness of an individual.
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:
        return 0  # Avoid selecting no features

    X_selected = X.iloc[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)

    # Apply LDA using the apply_lda function
    accuracy_lda = apply_lda(pd.DataFrame(X_scaled), y, data['neuropsych_score'])  # Pass scaled data as DataFrame

    return accuracy_lda  # Use only LDA accuracy here


def crossover(parent1, parent2):
    # Performs crossover at a random point.
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


def mutation(individual, indpb=0.05):
    # Performs flip-bit mutation at a single random point.
    if random.random() < indpb:  # Check if mutation should occur
        point = random.randint(0, len(individual) - 1)  # Select a random point
        individual[point] = 1 - individual[point]  # Flip the bit at the selected point
    return (individual,)


# Main Genetic Algorithm Call Function
def main(X, y, pop_size=100, ngen=10, cxpb=0.5, mutpb=0.2, top_percent=0.10, tolerance=1e-4):
    random.seed(42)

    # Initialize population
    population = [create_individual(n_features) for _ in range(pop_size)]

    # Evaluate initial population
    fitnesses = [evaluate(ind, X, y) for ind in population]

    # Store best individuals, their generation, and their fitness values
    best_individuals = []
    best_generations = []
    best_fitnesses = []

    for gen in range(ngen):
        # Selection (Select top individuals directly)
        num_top = int(pop_size * top_percent)
        top_indices = np.argsort(fitnesses)[-num_top:]
        offspring = [population[i] for i in top_indices]

        # Crossover to create the rest of the offspring
        num_crossovers = pop_size - num_top  # Number of crossovers needed
        for _ in range(num_crossovers // 2):  # // 2 because each crossover creates 2 offspring
            parent1, parent2 = random.sample(offspring, 2)  # Select parents from the top individuals
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        # Mutation
        for i in range(num_top, len(offspring)):  # Mutate only the new offspring
            if random.random() < mutpb:
                offspring[i], = mutation(offspring[i])

        # Evaluate offspring
        fitnesses = [evaluate(ind, X, y) for ind in offspring]

        # Replace population with offspring (Elitism: Keep top individuals)
        population[:] = offspring  

        # Store the top individuals and their generation
        top_indices = np.argsort(fitnesses)[-num_top:]
        for i in top_indices:
            best_individuals.append(population[i])
            best_generations.append(gen)  # Store the generation

        # Store fitness of the best individual in the current generation
        best_fitnesses.append(max(fitnesses))

        # Convergence check
        if gen > 1 and abs(best_fitnesses[-1] - best_fitnesses[-2]) < tolerance:
            print(f"Converged at generation {gen}")
            break

    return best_individuals, best_generations, best_fitnesses  # Return all three lists

# Run the GA
best_individuals, best_generations, best_fitnesses = main(X, y, pop_size=100, top_percent=0.10)  # Use top 10%

# Print the final best solution and its generation
best_individual = max(best_individuals, key=lambda ind: evaluate(ind, X, y))
best_index = best_individuals.index(best_individual)
best_generation = best_generations[best_index]

selected_features_lda = [i for i, bit in enumerate(best_individual) if bit == 1]
print(f"Final Best Solution (found at generation {best_generation}):")
print(f"  Selected Features: {X.columns[selected_features_lda].tolist()}")
print(f"  Number of features selected: {len(selected_features_lda)}")

# Print fitness values across generations
print("\nFitness values across generations:")
for gen, fitness in enumerate(best_fitnesses):
    print(f"Generation {gen}: {fitness}")

Converged at generation 3
Final Best Solution (found at generation 2):
  Selected Features: ['general white matter', 'general grey matter', 'general csf', 'brainstem', 'putamen+pallidum', 'hippocampus+amygdala', 'left cerebral white matter', 'left lateral ventricle', 'left inferior lateral ventricle', 'left cerebellum white matter', 'left pallidum', '3rd ventricle', 'left hippocampus', 'left amygdala', 'left accumbens area', 'right cerebral white matter', 'right inferior lateral ventricle', 'right cerebellum cortex', 'right thalamus', 'right caudate', 'right pallidum', 'right amygdala', 'right ventral DC', 'ctx-lh-bankssts', 'ctx-lh-fusiform', 'ctx-lh-isthmuscingulate', 'ctx-lh-lateralorbitofrontal', 'ctx-lh-lingual', 'ctx-lh-medialorbitofrontal', 'ctx-lh-middletemporal', 'ctx-lh-parahippocampal', 'ctx-lh-parsopercularis', 'ctx-lh-parstriangularis', 'ctx-lh-precentral', 'ctx-lh-precuneus', 'ctx-lh-superiorparietal', 'ctx-lh-superiortemporal', 'ctx-lh-frontalpole', 'ctx-lh-transversetem

## Key Changes in the code  

### LDA Implementation without Data Leakage Fix:

The initial code applied LDA to the entire dataset before cross-validation, potentially causing data leakage.

The apply_lda function was introduced to perform LDA and Logistic Regression within each cross-validation fold, preventing data leakage and providing more accurate fitness evaluations.

### Mutation at a Random Point:

Initially, the mutation operation flipped bits of all features in a selected individual with a certain probability.

The mutation operation was modified to select a single random point (feature) in the individual and flip the bit at that point. This introduces more focused changes during mutation.

### Selection:

The initial code used tournament selection to randomly choose individuals for the next generation.

Elitism selection was introduced, where the top-performing individuals from each generation are directly passed on to the next generation, ensuring the preservation of high-quality solutions.

### Crossover from Best Individuals:

Initially, crossover was performed on randomly selected individuals from the population.

The crossover operation was modified to select parents only from the top-performing individuals, promoting the combination of good traits and potentially creating even better offspring.

The original top-performing individuals are preserved in the next generation, and crossover is used to create new offspring to fill the rest of the population.

### Fitness Tracking and Convergence Check:

The best_fitnesses list was added to store the fitness value of the best individual in each generation.

A convergence check was introduced to stop the genetic algorithm when the fitness value converges (i.e., the change in fitness between generations becomes very small).

The code now prints the fitness values across generations to show how the fitness improves over time.